**no care was taken to make sure this is theoretically/quantitatively correct** for a first pass. Produces uneven illumination (or uneven autofluorescence/background depending on how it's modified) and adds some shot-like noise on top of this to corrupt the signal. The goal is to simulate lower signal data, since our real starting sample data has high contrast and high SNR. 

In [137]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import scipy.misc
import random
from tifffile import imsave

In [138]:
#parameters to be used later
illumination_unevenness = 0.8 #modifies the sigma determining the illumination artifact
illumination_unevenness_power = 4
mean_dark_current = 10**2 
percent_dead_pix = 0.1
bit_depth_save = 16
autofluorescence = 0.05 #autofluorescence relative to signal

In [139]:
from skimage.io import imread
raw = imread("../data/confocal_drug_panel/DMSO_raw.tif")

In [140]:
raw.shape

In [141]:
im = raw[3,:,:,2]
plt.imshow(im, vmin = 0, vmax = im.max()*0.5)

In [142]:
mid_index = int(raw.shape[1]/2)
print(mid_index)

In [143]:
#crudely simulate uneven illumination

#get parameters that are the same across all of the channels and slices
mid_index = int(raw.shape[1]/2)
dim1 = raw.shape[1]
dim2 = raw.shape[2]
noisy = np.zeros(raw.shape)

def gaussian(x, mu, sigma):
            return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
mu = mid_index
sigma = mid_index*illumination_unevenness
gauss = gaussian(np.linspace(0, im.shape[0]-1, im.shape[1]), mu, sigma)
        
for Z in range(0,raw.shape[0]):
    for ch in range(0,raw.shape[3]):
        im = raw[Z,:,:,ch]
        noiseD1 = np.zeros([dim1,dim2]) 
        for x in range(0, dim1):
            noiseD1[:,x] = gauss
        illumination = np.power(noiseD1.transpose()*noiseD1 , illumination_unevenness_power/2)
        noisy[Z,:,:,ch] = illumination*im
        
plt.imshow(noisy[3,:,:,2], vmin = 0, vmax = im_illum_artifact.max()*0.5)

In [144]:
#add dark/thermal noise, uniform across image and normally distributed
#not technically correct, noise should come from dark current shot noise
for Z in range(0,raw.shape[0]):
    for ch in range(0,raw.shape[3]):
        im = raw[Z,:,:,ch]    
        dark_current = np.random.normal(mean_dark_current, 1, [dim1,dim2])
        noisy[Z,:,:,ch] = noisy[Z,:,:,ch] + dark_current


In [145]:
#pretend this is a ccd or scmos and add some dead pixels
num_dead = int(percent_dead_pix*dim1*dim2/100)

for Z in range(0,raw.shape[0]):
    for ch in range(0,raw.shape[3]):
        idxs = random.sample(list(np.arange(dim1*dim2)),num_dead)
        idx_bool = np.zeros([1,(dim1)*(dim2)], dtype=bool)
        idx_bool[0,np.array(idxs)] = True
        idx2d_bool = np.reshape(idx_bool, (dim1, dim2))
        noisy[Z,idx2d_bool,ch] = 0

In [146]:
plt.imshow(noisy[3,300:500,300:500,2], vmin = 0, vmax = noisy.max()*0.01)

In [147]:
noisy = np.uint16(noisy/noisy.max() * (2**bit_depth_save - 1 ))
imsave("../data/confocal_drug_panel/DMSO.tif",noisy)
noisy.shape

In [154]:
#write json file for the sample modified dataset
import json
meta = {'cell_type': 'fibroblast', 'channels': ["your_fav_protein", "nucleus", "actin"], 'pixel_size': [0.3459442,0.3459442,0.7694383], 'axes': ['ZXYTC'],'raw_data_date': ['add_this'], 'image_preprocessing_done': ["Noise_generator.ipynb", "Branch=day3-day4-rework", "GitCommitId=add_this"]}
with open('../data/confocal_drug_panel/DMSO_metadata.json', 'w') as f:
    json.dump(meta, f)